## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.spatial.distance import cdist
import warnings
import import_ipynb
from Utility import *

warnings.simplefilter(action='ignore', category=FutureWarning)
import random

warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans

importing Jupyter notebook from Utility.ipynb


C:\Users\shris\anaconda3\lib\site-packages\nbformat\__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


 ### Read Music Data by artists

In [ ]:
data = pd.read_csv(r"datasets/data_by_artist.csv")
data.head()

We only need **'valence', 'artists', 'count', 'popularity'** fields from the dataset. Hence, retrieving those fields and saving it in a csv file named as "artist_data.csv"


In [ ]:
artist_data = data[['valence', 'artists', 'count', 'popularity']]
artist_data.to_csv('datasets/artist_data.csv')

 ### Read Music Data from spotify dataset

In [ ]:
p = 0.02  # to randomly select 1% of the rows
df_playlist_spotify = pd.read_csv(r"datasets/spotify_dataset.csv", error_bad_lines=False, warn_bad_lines=False,
                                  skiprows=lambda i: i > 0 and random.random() > p)
df_playlist_spotify.head()

### Clean up column names
1. remove extra quotes 
2. update column names by removing **name** from column name
3. remove extra white spaces

In [ ]:
df_playlist_spotify.columns = df_playlist_spotify.columns.str.replace('"', '')
df_playlist_spotify.columns = df_playlist_spotify.columns.str.replace('name', '')
df_playlist_spotify.columns = df_playlist_spotify.columns.str.replace(' ', '')
df_playlist_spotify.columns

### For recommender system, we are only keeping the artists with frequency higher than 50

In [ ]:
df_playlist = df_playlist_spotify.groupby('artist').filter(lambda x: len(x) >= 50)

### Only keep users with at least 10 unique artists in their playlists in order to reduce the impact of cold start problem

In [ ]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]

### Group by to get the frequency count for each user and artist (number of times that an artist has appeared in playlists created by a user)

In [ ]:
size = lambda x: len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0: 'freq'})[
    ['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

### Create a DF for artists and add artist id

In [ ]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index': 'artist_id', 0: 'artist'})
df_artist.head()

## Hybrid Song Recommendation 

It is a recommendation system which is combination of the content and collaborative filtering method.  By using content and collaborative-based methods to generate predictions separately and then combining the prediction, we can generate more accurate recommendations and also reduces the problem of cold start.

1. **PCA_algorithm**: to reduce the dimensionality (in our case number of components is 2) within a dataset while still retaining as much information as possible.

2. **KMeans_with_PCA_algorithm**: algorithm that tries to partition the dataset into Kpre-defined distinct non-overlapping clusters. 
    Based on the values of the Within Cluster Sum of Squares (WCSS) and an approach known as the Elbow method, we made a decision about how many clusters we’d like to keep.In our case, we have kept number of cluster as 50.

3. we have then created a new data frame df_segm_pca_kmeans. It allows us to add in the values of the separate components to our segmentation data set. The components’ scores are stored in the ‘scores P C A’ variable. Let’s label them com1 and com2.

4. **filter_based_on_segment**: filters songs that belong to particulat cluster segment.

5. **filter_based_on_cluster_centroid**: filters songs based on closet distance of song and centroid

6. returns list of top n artists, where n is set as 10 in our case

In [ ]:
def recommend_artists(artist,  n=10):    
    """
    Recommends songs based on a list of previous songs that a user has listened to.
    """
    song_embedding = PCA_algorithm(artist_data)
    kmeans_pca, centroids = KMeans_with_PCA_algorithm(song_embedding,50)

    df_segm_pca_kmeans = pd.concat([artist_data.reset_index(drop=True), pd.DataFrame(song_embedding)], axis=1)
    df_segm_pca_kmeans.columns.values[-2:] = ['com1', 'com2']
    df_segm_pca_kmeans['Segment K-means PCA'] = kmeans_pca.labels_

    #get segment value of artist
    segment_val = df_segm_pca_kmeans[df_segm_pca_kmeans['artists'] == artist]['Segment K-means PCA'].values[0]

    filtered_data_per_segment = filter_based_on_segment(df_segm_pca_kmeans, segment_val,'artists', 'valence')

    rec_artists = filter_based_on_cluster_centroid(kmeans_pca, filtered_data_per_segment, segment_val, artist_data)

    #recommend top n artists
    return rec_artists.head(n)['artists'].tolist()

**recommend_artist_and_songs:** Finds list of songs and artists  based on the selected artist name 
Recommends Top N artist with their song(s). In  our case, we have recommended top 10 songs

In [ ]:
def recommend_artist_and_songs(artist, n_songs=10):
    # get recommended artists
    recommended_artists = pd.DataFrame({
            'artist' : recommend_artists(artist, n_songs)
        })
    recommend_songs_list = pd.DataFrame(columns = ['artist', 'songs'])

    # get songs per artist
    for i in range(len(recommended_artists)):
        artist = recommended_artists.loc[i, "artist"]
        recommend_song = pd.DataFrame({
            'artist' : artist,
            'songs' :get_song_from_artist(artist, 1)
        })
        recommend_songs_list = recommend_songs_list.append(recommend_song, ignore_index = True)
    return  recommend_songs_list

**pass favorite artist name and no of recommended artist/song you want to listen and it gives you the list with n recommended artist with 1 songs per artist**

In [ ]:
recommended_artist_and_songs = recommend_artist_and_songs('Ella Fitzgerald',10)
print(recommended_artist_and_songs)

## Circle graph to show recommended songs

In [ ]:
circlify_vizualization(recommended_artist_and_songs['songs'])

## Circle graph to show recommended Artists

In [ ]:
circlify_vizualization(recommended_artist_and_songs['artist'])